In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import gensim

from random import randrange
import re
% matplotlib inline

/home/yuqian/.pyenv/versions/3.6.5/envs/env3.6/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/yuqian/.pyenv/versions/3.6.5/envs/env3.6/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
UsageError: Line magic function `%` not found.


In [2]:
labeled = pd.read_csv("data/labeled_data.csv")
unlabeled = pd.read_csv("data/unlabeled_data.csv")

test = pd.read_csv("data/test_data.csv")
test_label = pd.read_csv("data/test_label.csv")



In [3]:
# Check if there exists any nan value
labeled.isnull().values.any(),unlabeled.isnull().values.any()

(False, False)

In [4]:
labeled.head()

,text,label
0,The new rule is - \r\nif you are waiting for a...,4
1,"Flirted with giving this two stars, but that's...",3
2,I was staying at planet Hollywood across the s...,5
3,Food is good but prices are super expensive. ...,2
4,Worse company to deal with they do horrible wo...,1


In [5]:
# clean data for preprocessing functions

# Case Norm  
# Tokenize: since I will use embeddings, it's better use gensim tokenize rather than NLTK
# stop word: "not" in stop word lists, which could significantly affect sentiment classification. eg. "good" vs. "not good"
# Removing by frequency

# need further modify with checking real data
# check token frequency

def remove_symble(x):
    # remove '\r'
    return x.replace('\r',' ').replace('\n',' ')

def deal_with_oov(x):
    # if using pre-trained embedding, dealing with oov
    pass

def remove_stopword(x, stopword = None):
    # Caution, don't remove words like ..NO, NOT, etc
    pass

def clean_data(x):
    x = remove_symble(x)
    
    return x

In [6]:
# generate TF-IDF
def generate_tfdf(x):
    pass

def generate_embedding(x, pre_trained):
    pass

In [7]:
def preprocessing(df):
    df['clean_text'] = df['text'].apply(clean_data)
    return df

labeled = preprocessing(labeled)
unlabeled = preprocessing(unlabeled)
labeled.head()

,text,label,clean_text
0,The new rule is - \r\nif you are waiting for a...,4,The new rule is - if you are waiting for a t...
1,"Flirted with giving this two stars, but that's...",3,"Flirted with giving this two stars, but that's..."
2,I was staying at planet Hollywood across the s...,5,I was staying at planet Hollywood across the s...
3,Food is good but prices are super expensive. ...,2,Food is good but prices are super expensive. ...
4,Worse company to deal with they do horrible wo...,1,Worse company to deal with they do horrible wo...


In [8]:
rand_pick = randrange(len(labeled))
labeled['text'][0], labeled['clean_text'][0],labeled['text'][rand_pick], labeled['clean_text'][rand_pick]

('The new rule is - \r\nif you are waiting for a table (which you almost always are) , you cant wait inside. \r\nThey just posted a sign upfront that it causes some concerns for the seated patrons. \r\nHow awful is that? \r\n\r\nI like that they included the apology along with "especially now in the cold". \r\n\r\np.s you can try calling in ahead to reserve a table but thats ONLY if the waiting list is short. Otherwise, you have to show up to reserve. Boourns. \r\n\r\n\r\nThis place could do no wrong in my eyes. Rattle away you equally-clever/witty-name-for a hot beverage.\r\n\r\nMust mention - I am obsessed with Mad Gab. \r\n\r\nxoxo\r\nM.',
 'The new rule is -   if you are waiting for a table (which you almost always are) , you cant wait inside.   They just posted a sign upfront that it causes some concerns for the seated patrons.   How awful is that?     I like that they included the apology along with "especially now in the cold".     p.s you can try calling in ahead to reserve a t

## TF-IDF
> Using gensim
!pip install gensim

In [10]:
unlabeled.head()

,text,clean_text
0,Had a good experience when my wife and I sat a...,Had a good experience when my wife and I sat a...
1,On my first to Montreal with my gf we came her...,On my first to Montreal with my gf we came her...
2,One of our favorite places to go when it's col...,One of our favorite places to go when it's col...
3,"The doctor was very nice, got in in a good amo...","The doctor was very nice, got in in a good amo..."
4,The Nook is an immediate phoenix staple! I ca...,The Nook is an immediate phoenix staple! I ca...


In [11]:
# use gensim simple_preprocessing: include case Norm
label_texts = [gensim.utils.simple_preprocess(i) for i in labeled['clean_text']]
unlabel_texts = [gensim.utils.simple_preprocess(i) for i in unlabeled['clean_text']]
texts = label_texts + unlabel_texts

In [12]:
dictionary = gensim.corpora.Dictionary(texts)
dictionary.save('./serialize/texts.dict')
corpus = [dictionary.doc2bow(text) for text in texts]
gensim.corpora.MmCorpus.serialize('./serialize/texts.mm', corpus)
tfidf = gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [21]:
tfidf.save("./serialize/model.tfidf") #save 
tfidf = models.TfidfModel.load("./model.tfidf") #next time just load 

In [22]:
#len([print(k,v) for k, v in corpus_tfidf[0]])

In [23]:
# vocab size 
len(dictionary)

178811

In [ ]:
def construct_matrix(length, values):
    vector = [0] * length
    for k, v in values:
        vector[k] = v

    return vector

sparse_tfidf = []
for sentence in corpus_tfidf:
    sparse_tfidf.append(construct_matrix(len(dictionary), sentence))